In [1]:
import pandas as pd
import json
from ast import literal_eval
from datetime import datetime
import gzip
import math


#### Importing users json file

In [2]:
with gzip.GzipFile('/Users/daya/Desktop/Pay/FR/users.json.gz','r',) as fin:
    data =[]
    for line in fin:
        data.append(json.loads(line.decode('utf-8')))


In [3]:
users_df = pd.json_normalize(data)

In [4]:
users_df.head()

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


In [5]:
users_df = users_df.rename({'_id.$oid': 'id', 'createdDate.$date':'createdDate', 'lastLogin.$date': 'lastLogin'}, axis=1)

#### Function to convert dates

In [6]:
def convert_date(x):
    if(math.isnan(x)):
        return(None)
    else:
        try:
            return(datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S'))
        except TypeError:
            return(None)

In [7]:
users_df['createdDate'] = users_df['createdDate'].apply(lambda x:convert_date(x))
users_df['lastLogin'] = users_df['lastLogin'].apply(lambda x:convert_date(x))

#### Final transformed users dataframe

In [8]:
users_df.head()

,active,role,signUpSource,state,id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30,2021-01-03 15:25:30
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37


In [9]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   active        495 non-null    bool  
 1   role          495 non-null    object
 2   signUpSource  447 non-null    object
 3   state         439 non-null    object
 4   id            495 non-null    object
 5   createdDate   495 non-null    object
 6   lastLogin     433 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


In [10]:
users_df.isna().sum()

active           0
role             0
signUpSource    48
state           56
id               0
createdDate      0
lastLogin       62
dtype: int64

In [11]:
# Check for duplicate records

users_df[users_df.duplicated()]

,active,role,signUpSource,state,id,createdDate,lastLogin
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
5,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
8,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04,2021-01-03 15:25:37
...,...,...,...,...,...,...,...
490,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22,2021-03-05 16:52:23
491,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22,2021-03-05 16:52:23
492,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22,2021-03-05 16:52:23
493,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22,2021-03-05 16:52:23


#### There are a large number of duplicate records of users.

In [12]:
# Finding the sources of signups

users_df['signUpSource'].unique()

array(['Email', 'Google', nan], dtype=object)

#### Data quality issues in Users:

##### More than 50% user records are duplicate. 

In [13]:
users_df.to_excel('/Users/daya/Desktop/Pay/FR/users.xlsx')

#### Importing Brands json

In [14]:
with gzip.GzipFile('/Users/daya/Desktop/Pay/FR/brands.json.gz','r',) as fin:
    data =[]
    for line in fin:
        data.append(json.loads(line.decode('utf-8')))

In [15]:
brands_df = pd.json_normalize(data)

In [16]:
brands_df.head()

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [17]:
brands_df = brands_df.rename({'_id.$oid': 'id', 'cpg.$id.$oid':'cpg_oid', 'cpg.$ref': 'cpg_ref'}, axis=1)

In [18]:
brands_df.shape

(1167, 9)

In [19]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   id            1167 non-null   object
 6   cpg_oid       1167 non-null   object
 7   cpg_ref       1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [20]:
# Checking missing values

brands_df.isna().sum()

barcode           0
category        155
categoryCode    650
name              0
topBrand        612
id                0
cpg_oid           0
cpg_ref           0
brandCode       234
dtype: int64

In [21]:
# Calculatin percentage of missing values 
brands_df.isna().mean() * 100

barcode          0.000000
category        13.281919
categoryCode    55.698372
name             0.000000
topBrand        52.442159
id               0.000000
cpg_oid          0.000000
cpg_ref          0.000000
brandCode       20.051414
dtype: float64

#### It shows that more than 50% of brands are missing category code.

In [22]:
# Check for duplicate records in brands
brands_df[brands_df.duplicated()]

,barcode,category,categoryCode,name,topBrand,id,cpg_oid,cpg_ref,brandCode


#### There are no duplicate records found in brands

In [23]:
# Finding different categories of brands

brands_df['category'].unique()

array(['Baking', 'Beverages', 'Candy & Sweets', 'Condiments & Sauces',
       'Canned Goods & Soups', nan, 'Magazines', 'Breakfast & Cereal',
       'Beer Wine Spirits', 'Health & Wellness', 'Beauty', 'Baby',
       'Frozen', 'Grocery', 'Snacks', 'Household', 'Personal Care',
       'Dairy', 'Cleaning & Home Improvement', 'Deli',
       'Beauty & Personal Care', 'Bread & Bakery', 'Outdoor',
       'Dairy & Refrigerated'], dtype=object)

In [24]:
# Finding category wise percentages of brands

brands_df['category'].value_counts() /len(brands_df) * 100

Baking                         31.619537
Beer Wine Spirits               7.712082
Snacks                          6.426735
Candy & Sweets                  6.083976
Beverages                       5.398458
Health & Wellness               3.770351
Magazines                       3.770351
Breakfast & Cereal              3.427592
Grocery                         3.341902
Dairy                           2.827763
Condiments & Sauces             2.313625
Frozen                          2.056555
Personal Care                   1.713796
Baby                            1.542416
Canned Goods & Soups            1.028278
Beauty                          0.771208
Deli                            0.514139
Cleaning & Home Improvement     0.514139
Beauty & Personal Care          0.514139
Bread & Bakery                  0.428449
Dairy & Refrigerated            0.428449
Household                       0.428449
Outdoor                         0.085690
Name: category, dtype: float64

It is observed that baking is the top category among the brands

#### Data Quality Issues in  Brands:

##### More than 50% of missing values in categoryCode is a significant data quality issue.

In [25]:
brands_df.to_excel('/Users/daya/Desktop/Pay/FR/brands.xlsx')

#### Importing Receipts json

In [26]:
with gzip.GzipFile('/Users/daya/Desktop/Pay/FR/receipts.json.gz','r',) as fin:
    data =[]
    for line in fin:
        data.append(json.loads(line.decode('utf-8')))

In [27]:
receipts_df = pd.json_normalize(data)

In [28]:
receipts_df.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [29]:
receipts_df.shape

(1119, 15)

In [30]:
receipts_df = receipts_df.explode('rewardsReceiptItemList')
receipts_df.reset_index(inplace=True)

In [31]:
receipts_df = receipts_df.fillna({'rewardsReceiptItemList':'{}'})
receipts_df['rewardsReceiptItemList'] = receipts_df['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts_df['rewardsReceiptItemList'] = receipts_df['rewardsReceiptItemList'].apply(literal_eval)

In [32]:
receipts_df_norm = pd.json_normalize(receipts_df['rewardsReceiptItemList'], errors='ignore', record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

In [33]:
receipts_df_final = pd.merge(receipts_df, receipts_df_norm, left_index=True,right_index=True, how='outer')

In [34]:
receipts_df_final.shape

(7381, 50)

In [35]:
# Check missing values in receipts

receipts_df_final.isna().sum()

index                                                           0
bonusPointsEarned                                            1401
bonusPointsEarnedReason                                      1401
pointsEarned                                                 1128
purchasedItemCount                                            484
rewardsReceiptItemList                                          0
rewardsReceiptStatus                                            0
totalSpent                                                    435
userId                                                          0
_id.$oid                                                        0
createDate.$date                                                0
dateScanned.$date                                               0
finishedDate.$date                                           1411
modifyDate.$date                                                0
pointsAwardedDate.$date                                      1301
purchaseDa

In [36]:
receipts_df_final.isna().mean()*100

index                                                         0.000000
bonusPointsEarned                                            18.981168
bonusPointsEarnedReason                                      18.981168
pointsEarned                                                 15.282482
purchasedItemCount                                            6.557377
rewardsReceiptItemList                                        0.000000
rewardsReceiptStatus                                          0.000000
totalSpent                                                    5.893510
userId                                                        0.000000
_id.$oid                                                      0.000000
createDate.$date                                              0.000000
dateScanned.$date                                             0.000000
finishedDate.$date                                           19.116651
modifyDate.$date                                              0.000000
points

Observations on data quality:
1. pointsEarned are missing in approximately 15% of records. If no points were earned then it should have been 0 but missing values indicate that the data for points earned was not recorded.
2. puchaseDate, purchasedItemCount is missing for about 6% of records. It could pose an issue if rewards are claimed post purchase to validate if the rewards were applicable during the purchase.
3. rewardsReceiptItemList.barcode is missing for about 58% records. It can be a major issue since we can't identify a product purchased without barcode.

In [37]:
receipts_brandCode = receipts_df_final.dropna(subset=['rewardsReceiptItemList.brandCode'])

In [38]:
brands_brandCode = brands_df.dropna(subset=['brandCode'])

In [39]:
brands_brandCode

,barcode,category,categoryCode,name,topBrand,id,cpg_oid,cpg_ref,brandCode
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
5,511111719885,Baking,BAKING,test brand @1612366146091,False,601ac142be37ce2ead43755b,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146091
...,...,...,...,...,...,...,...,...,...
1161,511111403845,Beer Wine Spirits,NaN,Blue Moon,False,5332f709e4b03c9a25efd0f2,5332f709e4b03c9a25efd0f1,Cogs,BLUE MOON
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs,LIPTON TEA Leaves


In [40]:
unmapped_brands = receipts_brandCode[~(receipts_brandCode['rewardsReceiptItemList.brandCode'].isin(brands_brandCode['brandCode']))]['rewardsReceiptItemList.brandCode'].sort_values().unique()

In [41]:
len(unmapped_brands)

186

There are about 186 products in receipts of brands which are not included in the brands dataset.

In [42]:
unmapped_users = receipts_df_final[~(receipts_df_final['userId'].isin(users_df['id']))]['userId'].unique()

In [43]:
len(unmapped_users)

117

There are about 117 users from receipts whose id doesnot exist in users dataset.

#### Third: Summary of data quality issues in the datasets
1. More than 50% user records are duplicate. 
2. More than 50% of missing values in categoryCode.
3. pointsEarned are missing in approximately 15% of records. If no points were earned then it should have been 0 but missing values indicate that the data for points earned was not recorded.
4. purchaseDate, purchasedItemCount is missing for about 6% of records. It could pose an issue if rewards are claimed post purchase to validate if the rewards were applicable during the purchase.
5. rewardsReceiptItemList.barcode is missing for about 58% records. It can be a major issue since we can't identify a product purchased without barcode.
6. There are about 186 products in receipts of brands which are not included in the brands dataset.
7. There are about 117 users from receipts whose id doesnot exist in users dataset.

In [44]:
receipts_df_final.to_excel('/Users/daya/Desktop/Pay/FR/receipts.xlsx')